In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME

RSEED = 42
# Modeling Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.dummy import DummyClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore')


## Objectives of this notebook
- Implement feature importance by using SHAP approach 
- Using the results of our random forest model 
- Data that are used: cleaned data of second iteration (including all missing values)

In [2]:
df = pd.read_csv('../data/Flu_Shot_Data_cleaned_2.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   26707 non-null  int64  
 1   h1n1_vaccine                 26707 non-null  int64  
 2   seasonal_vaccine             26707 non-null  int64  
 3   h1n1_concern                 26615 non-null  float64
 4   h1n1_knowledge               26591 non-null  float64
 5   behavioral_antiviral_meds    26636 non-null  float64
 6   behavioral_avoidance         26499 non-null  float64
 7   behavioral_face_mask         26688 non-null  float64
 8   behavioral_wash_hands        26665 non-null  float64
 9   behavioral_large_gatherings  26620 non-null  float64
 10  behavioral_outside_home      26625 non-null  float64
 11  behavioral_touch_face        26579 non-null  float64
 12  doctor_recc_h1n1             24547 non-null  float64
 13  doctor_recc_seas

In [4]:
# Column 'unnamed: 0' is another index and we will drop it 
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_vaccine                 26707 non-null  int64  
 1   seasonal_vaccine             26707 non-null  int64  
 2   h1n1_concern                 26615 non-null  float64
 3   h1n1_knowledge               26591 non-null  float64
 4   behavioral_antiviral_meds    26636 non-null  float64
 5   behavioral_avoidance         26499 non-null  float64
 6   behavioral_face_mask         26688 non-null  float64
 7   behavioral_wash_hands        26665 non-null  float64
 8   behavioral_large_gatherings  26620 non-null  float64
 9   behavioral_outside_home      26625 non-null  float64
 10  behavioral_touch_face        26579 non-null  float64
 11  doctor_recc_h1n1             24547 non-null  float64
 12  doctor_recc_seasonal         24547 non-null  float64
 13  chronic_med_cond

Converting features to numerical:

In [6]:
cleanup = {"age_group": {"18 - 34 Years": 1, "35 - 44 Years": 2, "45 - 54 Years": 3, "55 - 64 Years": 4,
                                  "65+ Years": 5},
            "education": {"< 12 Years": 1, "12 Years": 2, "Some College": 3, "College Graduate": 4},
            "race": {"White": 1, "Black": 2, "Hispanic": 3, "Other or Multiple": 4},
            "sex" : {"Female": 1, "Male": 2},
            "rent_or_own" : {"Own": 1, "Rent": 2},
            "hhs_geo_region" : {"lzgpxyit": 1, "fpwskwrf": 2, "qufhixun": 3, "bhuqouqj": 4, "oxchjgsf": 5, "kbazzjca": 6, "mlyzmhmf": 7, "atmpeygn": 8, "lrircsnp": 9, "dqpwygqj": 10},
            "census_msa" : {"MSA, Not Principle  City": 1, "MSA, Principle City": 2, "Non-MSA": 3},
            "income_poverty" : {"Below Poverty": 1, "<= $75,000, Above Poverty": 2, "> $75,000": 3},
            "employment_industry" : {"fcxhlnwr": 1, "wxleyezf": 2, "ldnlellj": 3, "pxcmvdjn": 4, "atmlpfrs": 5, "arjwrbjb": 6, "xicduogh": 7, "mfikgejo": 8, "vjjrobsf": 9,
                                    "rucpziij": 10, "xqicxuve": 11, "saaquncn": 12, "cfqqtusy": 13, "nduyfdeo": 14, "mcubkhph": 15, "wlfvacwt": 16, "dotnnunm": 17, "haxffmxo": 18, "msuufmds": 19, "phxvnwax": 20,
                                    "qnlwzans": 21},
           "employment_occupation" : {"xtkaffoo": 1, "mxkfnird": 2, "emcorrxb": 3, "cmhcxjea": 4, "xgwztkwe": 5, "hfxkjkmi": 6, "qxajmpny": 7, "xqwwgdyp": 8, "kldqjyjy": 9,
                                    "uqqtjvyb": 10, "tfqavkke": 11, "ukymxvdu": 12, "vlluhbov": 13, "oijqvulv": 14, "ccgxvspp": 15, "bxpfxfdn": 16, "haliazsg": 17, "rcertsgn": 18, "xzmlyyjv": 19, "dlvbwzss": 20,
                                    "hodpvpew": 21, "dcjcmpih": 22, "pvmttkik": 23},
           "marital_status" : {"Married": 1, "Not Married": 2},
           "employment_status" : {"Employed": 1, "Not in Labor Force": 2, "Unemployed": 3}
                                  }

In [7]:
df_num = df.replace(cleanup)

In [8]:
df_num_cols = df_num.columns

Filling in null values with a dummy value:

In [9]:
imp_const = SimpleImputer(strategy='constant', fill_value=-999)
df_num = imp_const.fit_transform(df_num)
df_num = pd.DataFrame(df_num, columns=df_num_cols)

## Setting up the model
- Preparing data 
- Encoding the categorical variables 
- Instatiating the Random Forest model 
- sklearn preprocessing cannot be uses because it does not fit with the SHAP implementation

## H1N1 prediction

Setting up target variables:

In [10]:
y_h1n1 = df_num['h1n1_vaccine'].copy()
y_h1n1 = y_h1n1.to_numpy()
y_h1n1

array([0., 0., 0., ..., 0., 0., 0.])

Setting up features:

In [11]:
X = df_num.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])

Setting up categorical features for preprocessor:

In [12]:
cat_features = list(X.columns)

In [13]:
cat_features

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'health_insurance',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children',
 'employment_industry',
 'employment_occupation']

Preprocessor:

In [14]:
# Pipeline for categorical features
# This stays the same for everything
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='error', drop='first'))
])

In [15]:
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features)
])

In [16]:
rand_forst= RandomForestClassifier()

Instantiate estimator:

In [17]:
rand_forst_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", rand_forst),
])

In [18]:
rand_forst_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('1hot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavioral_outside_home',
                                                   'behavioral_touch_face',
                                                   'doctor

Train test split for h1n1_vaccine:

In [19]:
X_train, X_test, y_h1n1_train, y_h1n1_test = train_test_split(X, y_h1n1, stratify = y_h1n1, test_size=0.2, random_state=RSEED)

In [20]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_h1n1_train:', y_h1n1_train.shape)
print('y_h1n1_test:', y_h1n1_test.shape)

X_train shape: (21365, 35)
X_test shape: (5342, 35)
y_h1n1_train: (21365,)
y_h1n1_test: (5342,)


Fitting model and predicting for H1N1 vaccine:

In [21]:
rand_forst_h1n1_model = rand_forst_pipeline.fit(X_train, y_h1n1_train)

In [22]:
# Predictions on train data
rand_forst_h1n1_train_pred = rand_forst_h1n1_model.predict(X_train)

# Predictions on test data
rand_forst_h1n1_test_pred = rand_forst_h1n1_model.predict(X_test)

Evaluation:

In [23]:
# H1N1 ROC train data
print(roc_auc_score(y_h1n1_train, rand_forst_h1n1_train_pred))

# H1N1 ROC test data
print(roc_auc_score(y_h1n1_test, rand_forst_h1n1_test_pred))

1.0
0.703702974589236


## Seasonal prediction

Setting up target variables:

In [24]:
y_seasonal = df_num['seasonal_vaccine'].copy()
y_seasonal = y_seasonal.to_numpy()
y_seasonal

array([0., 1., 0., ..., 1., 0., 0.])

Setting up features:

In [25]:
X = df_num.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])

Setting up categorical features for preprocessor:

In [26]:
cat_features = list(X.columns)

In [27]:
cat_features

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'health_insurance',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children',
 'employment_industry',
 'employment_occupation']

Preprocessor:

In [28]:
# Pipeline for categorical features
# This stays the same for everything
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='error', drop='first'))
])

In [29]:
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features)
])

In [30]:
rand_forst= RandomForestClassifier()

Instantiate estimator:

In [31]:
rand_forst_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", rand_forst),
])

In [32]:
rand_forst_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('1hot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavioral_outside_home',
                                                   'behavioral_touch_face',
                                                   'doctor

Train test split for seasonal vaccine:

In [33]:
X_train, X_test, y_seas_train, y_seas_test = train_test_split(X, y_seasonal, stratify = y_seasonal, test_size=0.2, random_state=RSEED)

In [34]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_seas_train:', y_seas_train.shape)
print('y_seas_test:', y_seas_test.shape)

X_train shape: (21365, 35)
X_test shape: (5342, 35)
y_seas_train: (21365,)
y_seas_test: (5342,)


Fitting model and predicting for seasonal vaccine:

In [35]:
rand_forst_seas_model = rand_forst_pipeline.fit(X_train, y_seas_train)

In [36]:
# Predictions on train data
rand_forst_seas_train_pred = rand_forst_seas_model.predict(X_train)

# Predictions on test data
rand_forst_seas_test_pred = rand_forst_seas_model.predict(X_test)

Evaluation:

In [37]:
# seasonal ROC train data
print(roc_auc_score(y_seas_train, rand_forst_seas_train_pred))

# seasonal ROC test data
print(roc_auc_score(y_seas_test, rand_forst_seas_test_pred))

1.0
0.7764883171828008


In [39]:
# seasonal ROC train data
print(accuracy_score(y_seas_train, rand_forst_seas_train_pred))

# seasonal ROC test data
print(accuracy_score(y_seas_test, rand_forst_seas_test_pred))

1.0
0.7794833395731936


In [40]:
# seasonal ROC train data
print(f1_score(y_seas_train, rand_forst_seas_train_pred))

# seasonal ROC test data
print(f1_score(y_seas_test, rand_forst_seas_test_pred))

1.0
0.7558043117744611


In [41]:
# seasonal ROC train data
print(precision_score(y_seas_train, rand_forst_seas_train_pred))

# seasonal ROC test data
print(precision_score(y_seas_test, rand_forst_seas_test_pred))

1.0
0.7800599058622165


In [42]:
# seasonal ROC train data
print(recall_score(y_seas_train, rand_forst_seas_train_pred))

# seasonal ROC test data
print(recall_score(y_seas_test, rand_forst_seas_test_pred))

1.0
0.7330116606353035


## Feature importance

**H1N1 feature importance with eli5**

In [38]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rand_forst_seas_model.steps[1][1], random_state=1).fit(X_test, y_seas_test)
eli5.show_weights(perm, top=None, feature_names = X_test.columns.tolist())

ValueError: X has 35 features, but DecisionTreeClassifier is expecting 148 features as input.

**Seasonal feature importance with eli5**

In [ ]:
seas_model = rand_forst_seas_model.steps[1][1]

In [ ]:
seas_model

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(seas_model, random_state=RSEED).fit(X_test, y_seas_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [ ]:
get_feature_names(preprocessor)